# Part 1: Scrape Wikipedia for Toronto neighborhoods

Import the necessary libraries

In [1]:
# import libraries
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

## Scrape the wiki page into a dataframe

Use requests to get the contents of the wiki page to be scraped and stuff it into a soup object

In [2]:
# Get contents of wiki page
wiki_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r  = requests.get(wiki_url)

# Create the soup object for parsing
data = r.content
soup = BeautifulSoup(data, "lxml")

Parse the contents of the file

In [3]:
# create the empty dataframe
column_names=['PostalCode', 'Borough', 'Neighborhood']
df_nb = pd.DataFrame(columns=column_names)

# pull the table out of the html
table=soup.find('table', {"class": "wikitable sortable"})

# parse the table rows into the dataframe
rows = table.find_all('tr')
for row in rows:
    dataitems = row.find_all('td')
    tr_data=[di.text.strip() for di in dataitems]
    if tr_data:
        df_nb.loc[len(df_nb)] = tr_data

## Clean up the dataframe

Clean up rows with "Not assigned" values

In [4]:
# Remove rows where Borough is Not assigned
df_nb = df_nb[df_nb['Borough'] != 'Not assigned'].reset_index(drop=True)

In [5]:
# make neighborhood the same as borough where neighborhood isn't assigned
df_nb.loc[df_nb['Borough'] == 'Not assigned'] = r"Queen's Park" # There's only one row like this so we can assign the value directly

Group rows that have the same neighborhood

In [6]:
# aggregate duplicate rows for the same neighborhood
postal_codes = df_nb.groupby(['PostalCode', 'Borough']).agg({
    'Neighborhood': lambda x: ', '.join(x)
})

In [7]:
# reset the index in order to match the dataframe format given in the problem
postal_codes.reset_index(inplace=True)

## Show the contents and size of the resulting dataframe

In [8]:
postal_codes.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
postal_codes.shape

# My response shows (103, 3)

(103, 3)

# Part 2: Add geospatial data to the postal codes

First, get the geo data

In [10]:
# Getting the data from the file provided instead of the suggested flaky API
geo_url = 'http://cocl.us/Geospatial_data'

lat_longs = pd.read_csv(geo_url)

In [11]:
lat_longs.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now to merge the two dataframes.

First we need to rename the Postal Code column to match our existing dataframe so that we can merge.

In [12]:
lat_longs.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
lat_longs.columns

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')

Looks good, now let's merge

In [13]:
postal_lat_longs = pd.merge(postal_codes, lat_longs, on='PostalCode')

## The final dataframe including the lat_longs

In [15]:
postal_lat_longs.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3 - Clustering of Toronto neighborhoods

## Overview

I decided to replicate the neighborhood clustering analysis from the lab.

### Setup

Import libraries for mapping and k-means analysis

In [18]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this if the import below fails.
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if the import below fails
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

Find the location of Toronto for centering the map.

In [19]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Make a map of central Toronto with the neighborhoods

First let's filter the dataframe to only "Toronto" boroughs

In [60]:
toronto_data = postal_lat_longs[postal_lat_longs['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Now make the map.

In [26]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12, width=900, height=600)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Get some data about the neighborhoods from Foursquare

Secret keys below.

In [27]:
# The code was removed by Watson Studio for sharing.

In [30]:
#CLIENT_ID = 'client-id' # your Foursquare ID
#CLIENT_SECRET = 'client-secret' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

Define a function for getting the nearby venues for a list of locations. Pulled this directly from the lab.

In [71]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now run the code above to go get the data from Foursquare.

In [72]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
print("... and done.")

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

### Analyze neighborhoods

In [73]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
2,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
3,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery


### Use onehot encoding to calculate the percentage of each kind of restaurant by neighborhood.

In [74]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
# calculate the weighted averages by neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Adelaide, King, Richmond",0.00,0.0,0.0,0.0000,0.0000,0.040000,0.0,0.0,0.0,...,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0
1,Berczy Park,0.00,0.0,0.0,0.0000,0.0000,0.010000,0.0,0.0,0.0,...,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.01,0.0,0.0,0.0000,0.0000,0.010000,0.0,0.0,0.0,...,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.00,0.0,0.0,0.0000,0.0000,0.021739,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0.0,0.0,0.0625,0.0625,0.000000,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


## Now run k-means clustering to find the most similar neighborhoods

In [76]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 1, 3, 0, 6, 1, 1, 3, 3, 5, 1, 5, 5, 0, 1, 3, 1, 0, 3, 1, 5, 3, 2,
       3, 0, 5, 3, 0, 4, 5, 1, 1, 1, 3, 3, 3, 5, 5], dtype=int32)

Put the clusters into our toronto_data dataframe.

In [77]:
# add clustering labels
toronto_data['Cluster Labels'] = kmeans.labels_

## Time to visualize the resulting clusters.

In [79]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12, width=900, height=600)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood'], toronto_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters